# Type inference

Because anything is strictly typed in C++, you have to specify the type of any variable when you declare it. This can be a bit cumbersome, especially when the type is long or complex. Yet, compilers have long been able to perform *type deduction* for function templates.

C++11 introduces a similar mechanism, called *type inference* based on keyword `auto`, which enables to infer the type of a variable from its initial value. The use of `auto` has progressively been enlarged to more and more situations.

## Keyword `auto` as a variable type

### Starting with C++11, the type of a variable can be deduced from its initial value

In [ ]:
std::map<std::vector<int>,std::list<float>> m ;
auto itr = m.begin() ;

### Possible `const` and `&` are dropped

In [ ]:
int const i = 2 ; // int const
auto j = i ;      // int
int & k = j ;     // int &
auto l = k ;      // int

### On the contrary, one can add `const`, `&` or `*`

In [ ]:
int i = 2 ;
auto & j = i ;
auto const & k = j ;
auto const * l = &k ;
auto const * const m = &l ;

## Keyword `decltype`

### In order to reuse the type of an expression 

In [ ]:
std::map<std::vector<int>,std::list<float>> collection1 ;
decltype(collection1) collection2 ;

### So to help type inference, when there is no initial value

In [ ]:
std::map<std::vector<int>,std::list<float>> collection ;
decltype(collection)::iterator itr ;

### So to avoid `const` and `&` dropping

In [ ]:
int i = 10 ;           // int
int & j = i ;          // int &
decltype(auto) k = i ; // int
decltype(auto) l = j ; // int &

## How to know which type has infered the compiler ?

* Your editor, if smart enough, may help.
* To be sure, you can also trigger an intentional compiler error, whose message contains the type name.

In [1]:
%%file tmp.inference.cpp

template <typename T> struct TypeDisplayer ;

int main()
 {
  int const i = 2 ;
  auto & j = i ;
  TypeDisplayer<decltype(j)> td ;
}

Writing tmp.inference.cpp


In [2]:
!rm -f tmp.inference.exe && g++ -std=c++17 tmp.inference.cpp -o tmp.inference.exe

tmp.inference.cpp: In function ‘int main()’:
tmp.inference.cpp:8:30: error: aggregate ‘TypeDisplayer<const int&> td’ has incomplete type and cannot be defined
    8 |   TypeDisplayer<decltype(j)> td ;
      |                              ^~


## Keyword `auto` in a range-based for

### What is a range-based for ?

For any collection which is supported by `std::begin()` and `std::end()`, one can now use the **ranged-based for** notation.

In [ ]:
#include <iostream>

int const MAX = 5 ;
double values[MAX] = { 1.1, 2.2, 3.3, 4.4, 5.5 } ;

for ( double value : values  )
 { std::cout << value << " " ; }

### Combined with `auto`

In [ ]:
#include <vector>
#include <iostream>

int i = 0 ;
double values[MAX] = { 1.1, 2.2, 3.3, 4.4, 5.5 } ;

for ( auto value : values  )
 { std::cout << value << " " ; }

### Modifications requires &

In [ ]:
#include <iostream>

int i = 0 ;
std::vector<int> values(5,0) ;

for ( auto & value : values  )
 { value = ++i ; }

for ( auto value : values  ) 
 { std::cout << value << " " ; }

## Keyword `auto` as function return type

### When used alone (C++14)

When `auto` is used instead of the usual return type, the compiler will infer this type from the return statements in the function body.

In [ ]:
auto nb( int i )
 {
  if (i<10) return 1 ;
  if (i<100) return 2 ;
  return 99 ;
 }

In [ ]:
nb(15)

### When used with a trailing return type (C++11)

The keyword `auto` can also be used in a *trailing return type* declaration, as shown below. There is no inference there. That is just a syntax to declare the type after the function parameters.

In [ ]:
%%file tmp.trailing.cpp

#include <iostream>

auto add( int p1, int p2 ) -> int
 { return (p1+p2) ; }
 
int main()
 {
  std::cout << add(39,3.14) << std::endl ;
 }

In [ ]:
!rm -f tmp.trailing.exe && g++ -std=c++11 tmp.trailing.cpp -o tmp.trailing.exe && ./tmp.trailing.exe

This can help when the return type depends on the parameter types:

In [15]:
%%file tmp.trailing.cpp

#include <iostream>

template< typename T1, typename T2 >
auto add( T1 value, T2 offset ) -> decltype(value+offset)
 { return (value+offset) ; }
 
int main()
 {
  std::cout << add(39,3.14) << std::endl ;
 }

Overwriting tmp.trailing.cpp


In [16]:
!rm -f tmp.trailing.exe && g++ -std=c++11 tmp.trailing.cpp -o tmp.trailing.exe && ./tmp.trailing.exe

42.14


## Structured Bindings (C++17)

The functional programming style should lead you to write "pure" functions, with constant input data, and all the ouput returned as a compound data. The new C++17 structured bindings enable you to dispatch such a compound return value into several variables, almost as simply as in Python...

In [7]:
%%file tmp.bindings.cpp

#include <iostream>
#include <utility>

int main() {
  auto [ first, second ] = std::make_pair(42, 3.14) ;
  std::cout << "first: " << first << std::endl ;
  std::cout << "second: " << second << std::endl ;
}

Writing tmp.bindings.cpp


In [8]:
!rm -f tmp.bindings.exe && g++ -std=c++17 tmp.bindings.cpp -o tmp.bindings.exe && ./tmp.bindings.exe

first: 42
second: 3.14


Actually, on the right side of `=`, one can place many kinds of arrays, structs, tuples, and user-defined classes with tuple-like properties.

In [9]:
%%file tmp.bindings.cpp

#include <iostream>

int main() {
  int values[] = { 1, 2 } ;
  auto [ v1, v2 ] = values ;
  std::cout << v1 << ", " << v2 << std::endl ;
}

Overwriting tmp.bindings.cpp


In [10]:
!rm -f tmp.bindings.exe && g++ -std=c++17 tmp.bindings.cpp -o tmp.bindings.exe && ./tmp.bindings.exe

1, 2


In [11]:
%%file tmp.bindings.cpp

#include <iostream>

struct Vector { double x, y, z ; } ;

int main() {
  Vector origin = { 0., 0., 0. } ;
  auto [ ox, oy, oz ] = origin ;
  std::cout << ox << ", " << oy << ", " << oz << std::endl ;
}

Overwriting tmp.bindings.cpp


In [12]:
!rm -f tmp.bindings.exe && g++ -std=c++17 tmp.bindings.cpp -o tmp.bindings.exe && ./tmp.bindings.exe

0, 0, 0


In [13]:
%%file tmp.bindings.cpp

#include <iostream>
#include <tuple>

int main() {
  std::tuple t(42, 3.14, "hello", "world") ;
  auto [ t1, t2, t3, t4 ] = t ;
  std::cout << t1 << ", " << t2 << ", " << t3 << ", " << t4 << std::endl ;
}

Overwriting tmp.bindings.cpp


In [14]:
!rm -f tmp.bindings.exe && g++ -std=c++17 tmp.bindings.cpp -o tmp.bindings.exe && ./tmp.bindings.exe

42, 3.14, hello, world


Structured bindings also prove useful within `range-based for`:

In [ ]:
%%file tmp.bindings.cpp

#include <iostream> 
#include <map> 
#include <string> 

int main() {

  std::map<std::string, int> grades ;
  
  grades["Francoise"] = 12 ;
  grades["Antoine"] = 18 ;
  grades["David"] = 3 ;
      
  for ( auto [ key, value ] : grades )
   { std::cout << key << " " << value << std::endl ; } 

}

In [ ]:
!rm -f tmp.bindings.exe && g++ -std=c++17 tmp.bindings.cpp -o tmp.bindings.exe

In [ ]:
!./tmp.bindings.exe

## Keyword `auto` as function parameter type (C++20)

When used as parameter type, `auto` is a shortcut for the template syntax. Yet, each parameter must be declared with `auto` individually, and those parameters are independants one from each other.

```c++
bool compare( auto param1, auto param2 )
 { return param1==param2 ; }
```

means:

```c++
template <typename T1, typename T2>
bool compare( T1 param1, T2 param2 )
 { return param1==param2 ; }
```

not:


```c++
template <typename T>
bool compare( T param1, T param2 )
 { return param1==param2 ; }
```


# Quizz : what is the return type ? 

```c++
auto join( std::vector<std::string> const & values, std::string separator )
 {
  if (std::empty(values))
   { return "" ; }
  auto result = values[0] ;
  for ( std::size_t i = 1 ; i < std::size(values) ; ++i )
   { result += separator ; result += values[i] ; } 
  return result ;
 }
```

# Take away

- The keyword `auto` basically avoid typing redundant types.
- The keyword `decltype` helps to keep the exact original type.
- Since C++20, `auto` is also a simplified syntax for simple templates.
- Type inference and template instanciation "mostly" follow the same rules...
- **Overuse of `auto` obfuscates the code!**

# Questions ?

Ask [Pythie++](https://app.corolair.com/student/tutor/a1076050-7a47-492f-b0af-bb09f551be73/chat) and|or try its quizz !

# Exercise

1. In the code below, simplify the friend operator `<<`.
2. Provide the class with methods `begin()` and `end()`.
3. Move the operator `<<` outside the class.

In [ ]:
%%file tmp.inference.cpp

#include <iostream>
#include <string>
#include <vector>

class Sentence
 {
  public :
    void add( char const * word )
     { m_words.push_back(static_cast<std::string>(word)) ; }
    friend std::ostream & operator<<( std::ostream & os, Sentence const & s )
     {
      typedef typename std::vector<std::string>::const_iterator Iterator ;
      for ( Iterator word = s.m_words.begin() ; word != s.m_words.end() ; ++word )
       { os<<(*word)<<" " ; }
      return os ;
     }
  private :
    std::vector<std::string> m_words ;
 } ;

int main()
 {
  Sentence s ;
  s.add("Hello") ;
  s.add("world") ;
  s.add("!") ;
  std::cout<<s<<std::endl ;
 }

In [ ]:
!rm -f tmp.inference.exe && g++ -std=c++17 tmp.inference.cpp -o tmp.inference.exe

In [ ]:
!./tmp.inference.exe

© *CNRS 2024*  
*This document was created by David Chamont and translated by Olga Abramkina. It is available under the [License Creative Commons - Attribution - No commercial use - Shared under the conditions 4.0 International](http://creativecommons.org/licenses/by-nc-sa/4.0/)*